In [1]:

####################################################################################################
####################################################################################################

import sys
import glob
import os.path

import datetime
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
#import seaborn as sns


from IPython.display import display
#from IPython.display import Image
from PIL import Image
#
#import tensorflow as tf
#
#print (tf.__version__)
#
#import keras
#import keras.utils.vis_utils
#from keras import backend as K
#from keras import layers
#from keras import regularizers
#from keras.layers import Dense
#from keras.layers import Dropout
#from keras.layers.normalization import BatchNormalization
#from keras.models import load_model
#from keras.models import Sequential
#from keras.utils import plot_model

#from xy_s2integrals_models import *
#from helpers import *

#sys.path.append(os.path.abspath("../"))
#from pax_utils import s1s2_utils

sys.path.append(os.path.abspath("../../"))
from pax_utils import waveform_utils



In [2]:
print()

####################################################################################################
# Input dataframe for all events
####################################################################################################

input_dir    = "../../pax_merge/merged/"
#input_file   = 'merged_pax_4k_test.pkl'
input_file   = 'merged_all_4k_test.pkl'

input_path   = input_dir + input_file
events_df     =  pd.read_pickle(input_path)

s2_window_max = np.amax( (events_df['event_s2_right']- events_df['event_s2_left']).as_matrix() ) + 1
s2_length_max = np.amax( events_df['event_s2_length'].as_matrix() )

assert(s2_window_max == s2_length_max)

print()
print("Event Max S2 Window size: " + str(s2_window_max))
print()

#print()
#for col in events_df.columns:
#    print(col)

#print(input_df.shape)
#display(input_df[:][:])
#print()


####################################################################################################
# Input dataframe for all events
####################################################################################################

#input_dir    = "../../pax_merge/merged/"
#input_file   = 'merged_pax_4k_test.pkl'
#input_path   = input_dir + input_file
#input_df     =  pd.read_pickle(input_path)

print()
#print(input_df.shape)
#display(input_df[0:1][:])
#print()
#print()






Event Max S2 Window size: 815




In [5]:

print()


####################################################################################################
# Input directory - containing S2 waveforms for all top channels
####################################################################################################

dir_input_s2    = "../../pax_merge/merged/waveforms_test/s2"
dir_format_s2   = dir_input_s2 + '/' + 'event' + ('[0-9]' * 4) + '_S2waveforms.pkl'
lst_contents_s2 = glob.glob(dir_format_s2)


####################################################################################################
# Input data shape: (1, 10, 127)
# Truth data shape: (1, 2)
####################################################################################################

nEvents           = len(lst_contents_s2)

train_data        = np.zeros((nEvents, s2_window_max, 127))
train_truth       = np.zeros((nEvents, 2))
event_train_data  = np.zeros((s2_window_max, 127))
event_train_truth = np.zeros(2)
                     
                             
####################################################################################################
# Loop over events
####################################################################################################

for iEvent, infile in enumerate(lst_contents_s2):
    
    ################################################################################################
    # Get Event Information
    ################################################################################################
    
    event_num = int(os.path.basename(infile).split("_")[0].replace("event", ""))
    
    df_event  = events_df.iloc[iEvent]
    event_s2_count  = df_event.loc['event_s2_count' ]
    event_s2_length = df_event.loc['event_s2_length']
    event_s2_left   = df_event.loc['event_s2_left'  ]
    event_s2_right  = df_event.loc['event_s2_right' ]
    intr_count      = df_event.loc['intr_count'     ]
    x_true          = df_event.loc['x'              ]
    y_true          = df_event.loc['y'              ]
    
    assert (event_s2_count  > 0)
    assert (event_s2_length > 0)
    assert (event_s2_left   > 0)
    assert (event_s2_right  > 0)

    print("iEvent: " + str(iEvent) + ", event_num: " + str(event_num))
    
    
    #display(events_df[0:1][0:10])
    
    
    ################################################################################################
    # Get S2 waveforms for all top channels
    ################################################################################################
    
    df_s2waveforms = pd.read_pickle(infile)
    df_s2waveforms = waveform_utils.addEmptyChannelsToDataFrame(df_s2waveforms)

      
    ################################################################################################
    ################################################################################################
    
    for iChannel, row in df_s2waveforms.iterrows():
        
        channel          = row['channel']
        channel_left     = row['left']
        channel_right    = row['right']
        channel_length   = row['length']
        channel_integral = row['sum']
        channel_raw_data = row['raw_data']
    
        assert(channel_length == channel_raw_data.size)
        
             
        ############################################################################################
        # Pad the S2 array for all channels in the event
        ############################################################################################
        
        arr_channel = np.zeros(event_s2_length)
        
        if (channel_length > 0):
            
            channel_left_offset  = channel_left   - event_s2_left
            channel_right_offset = event_s2_right - channel_right
            
            assert(channel_left    >= event_s2_left  )
            assert(channel_right   <= event_s2_right )
            assert(event_s2_length == channel_left_offset + channel_length + channel_right_offset)
            
            #print()
            #print("s2 window all chan:   " + str(event_s2_length))
            #print("channel length:       " + str(channel_length))
            #print("channel left offset:  " + str(channel_left_offset))
            #print("channel right offset: " + str(channel_right_offset))
            
            arr_channel[channel_left_offset : channel_left_offset + channel_length] = channel_raw_data  
        
            assert( abs(channel_integral - np.sum(arr_channel)) < 1e-4 )

        
        ############################################################################################
        # Pad to the widest S2 over all events
        ############################################################################################
            
        arr_channel_padded                       = np.zeros(s2_length_max)
        arr_channel_padded[0 : arr_channel.size] = arr_channel
        
         
        ############################################################################################
        ############################################################################################
        
        event_train_data[:, iChannel] = arr_channel_padded # timeseries for iChannel
        event_train_truth[0]          = x_true
        event_train_truth[1]          = y_true
    
        #print(arr_channel_padded.shape)
            
            
        continue

    
    ################################################################################################
    ################################################################################################
    
    train_data [0, :, :] = event_train_data
    train_truth[0, :   ] = event_train_truth
    
    
    ################################################################################################
    # Sanity
    ################################################################################################
    
    df_event_s2s  = df_s2waveforms[df_s2waveforms['left'] != 0].copy(deep=True)
    df_event_s2s.reset_index(inplace=True, drop=True)
    
    min_left      = np.amin( df_event_s2s.left.as_matrix()  )
    max_right     = np.amax( df_event_s2s.right.as_matrix() )
    max_length    = np.amax( df_event_s2s.length            )
  
    assert (min_left   <= event_s2_left  )
    assert (max_right  >= event_s2_right )
    assert (max_length <= event_s2_length)
    
    
    ################################################################################################
    ################################################################################################

    continue
    

####################################################################################################
####################################################################################################

#print()                       
#print("Event Input data shape: " + str(event_train_data.shape ))
#print("Event Truth data shape: " + str(event_train_truth.shape))
print()
print("Input data shape:       " + str(train_data.shape ))
print("Truth data shape:       " + str(train_truth.shape))
print()
        
print("Done")
    



iEvent: 0, event_num: 0
iEvent: 1, event_num: 1
iEvent: 2, event_num: 1000
iEvent: 3, event_num: 1001

Input data shape:       (4, 815, 127)
Truth data shape:       (4, 2)

Done


In [ ]:
    
    ################################################################################################
    ################################################################################################

    if (min_left < s2_left ):
        
        print()
        print("   S2 Event Left:            " + str(s2_left))
        print("   S2 Event Count:           " + str(s2_count))
        print("   S2 Waveform min left:     " + str(min_left)  )
        print("   S2 Waveform min left idx: " + str(min_left_idx)  )
    
        #print(df_event_s2s.shape)
        
        #display(df_event[0:10][:])
        
        display(df_min_left)
    
    
    ################################################################################################
    ################################################################################################
    
    if (max_right > s2_right ):
    
        print()
        print("   S2 Event Right:            " + str(s2_right))
        print("   S2 Waveform max right:     " + str(max_right) )
        print("   S2 Waveform max right idx: " + str(max_right_idx)  )
    

In [2]:

####################################################################################################
####################################################################################################

nEventsTrain = 100000
file_all     = "../data_lfs/merged_all_200k_1803261941.pkl"

df_all   = pd.read_pickle(file_all)
df_all   = df_all[df_all['intr_count'] == 1].reset_index(drop=True)
df_all   = df_all[0:nEventsTrain][:]


####################################################################################################
####################################################################################################

df_train_input = df_all[:][s1s2_utils.getS2integralsDataFrameColumns()]
df_train_truth = df_all[:][['x', 'y']]


####################################################################################################
####################################################################################################

checkTrainingData(df_train_input, df_train_truth)




Input shape: (100000, 127)
Truth shape: (100000, 2)

Input DataFrame: 


,s2_area_000,s2_area_001,s2_area_002,s2_area_003,s2_area_004,s2_area_005,s2_area_006,s2_area_007,s2_area_008,s2_area_009,...,s2_area_117,s2_area_118,s2_area_119,s2_area_120,s2_area_121,s2_area_122,s2_area_123,s2_area_124,s2_area_125,s2_area_126
0,1.715228,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.293108,...,6.564991,0.0,0.400128,2.742959,3.460256,8.159419,38.424745,15.890681,1.354377,10.354574
1,0.912039,0.0,0.0,1.545933,0.000000,0.000000,2.568431,0.000000,2.604376,2.010018,...,3.208600,0.0,0.000000,5.596787,26.363896,126.844246,55.087875,8.130994,8.979807,13.682349
2,0.000000,0.0,0.0,0.000000,2.104002,0.000000,1.731047,2.784006,0.000000,5.977608,...,3.487908,0.0,1.534094,0.000000,1.919893,4.920848,0.505347,2.435258,0.506562,4.203870
3,6.140486,0.0,0.0,9.062252,5.621141,1.789005,1.802042,0.000000,0.869444,0.000000,...,0.000000,0.0,0.918565,0.619911,0.000000,1.230013,0.788772,1.179353,2.046051,0.000000
4,4.333349,0.0,0.0,0.000000,1.313044,0.000000,0.613111,0.000000,1.061757,3.775456,...,12.939582,0.0,3.896350,2.259100,3.023539,0.855252,7.456719,1.257868,4.551362,5.000204



Truth DataFrame:


,x,y
0,2.626434,-17.960824
1,10.907536,-6.312277
2,24.631697,-12.763601
3,2.726006,37.749081
4,-18.078293,-23.212466


In [3]:

##########################################################################################
##########################################################################################

lst_models = [
    Xenon1TPosRecMLP(),
    bargeModel()
]



In [4]:

##########################################################################################
##########################################################################################

for iModel in range(0, len(lst_models)):
    
    ######################################################################################
    ######################################################################################

    model, model_name = lst_models[iModel]
    
    
    ######################################################################################
    ######################################################################################

    epochs     = 10
    batch_size = 64
    verbose    = 1
    
    
    ######################################################################################
    # Fit Model
    ######################################################################################
    
    print()
    print("Model Summary:")
    model.summary()
    print()
    #model.fit(df_train_input, df_train_truth, epochs=epochs, batch_size=batch_size, verbose=verbose)
    model.fit(df_train_input.values, df_train_truth.values, epochs=epochs, batch_size=batch_size, verbose=verbose)

    print("Training Complete!")

        
    ######################################################################################
    # Save Model
    ######################################################################################

    folder   = "models/"    
    name     = 'model' + '_' + model_name + '_' + 'elu' #model.activation
    name_h5  = folder + name + ".h5"
    name_png = folder + name + ".png"

    model.save(name_h5, overwrite=True)
        
   
    ######################################################################################
    ######################################################################################

    plot_model(model, to_file=name_png, show_layer_names=True, show_shapes=True)
    
    #Image(filename=name_png) 
    #display(Image.open(name_png))


    ######################################################################################
    ######################################################################################
    
    continue
    
    


Model Summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 127)               16256     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               12800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 80)                8080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 40)                3240      
_________________________________________________________________
dropout_3 (Dropout)          (None, 40)                0    